# Lecture 11a
. Naive Bayes Classification

Learn to find the implementation: [Scikit-learn,](https://scikit-learn.org/stable/#)
[Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)

## Dataset: iris flowers,
loaded from csv file on your disk, or from seaborn, or from scikit-learn   
[Data Source: UCI](https://archive.ics.uci.edu/ml/datasets/iris)

[Data Info](https://archive.ics.uci.edu/ml/datasets/iris)  


**Summary about the algo:**    
Naive Bayes is an "supervised learning" model.   
We know the labels. We have a "y" target variable. 

Goal:  
Classify observations in known categories.

Important note:   
Assumption of conditional independence between every pair of features.  

In [1]:
# use pwd command to see in which folder we are working in.
# pwd means present working directory
# you may need !pwd in Mac OS

In [2]:
pwd  

'C:\\Users\\tharg\\uoa_py_course\\lectures\\ipynb_files\\lecture_11_nbc_logregc\\nbc'

# A. Import the necessary modules.   
You may add libraries here as you go on with your work. But, all imports should be at the first code cell.

In [3]:
# data management libraries
import pandas as pd
import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# interactive visualizarion libaries
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# scikit learn algo library (google it)
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
# data preprocessing 
from sklearn.model_selection import train_test_split
# algo metric
from sklearn.metrics import silhouette_samples, silhouette_score, accuracy_score

# B. Load the data and understand its features.   
See which are the variables, what is their type, what are the values that the variables take.  
At this step, you should think about possible relations that you ought to examine.   
Which do you think might be more important?

In [4]:
# load iris from scikit-learn
iris = datasets.load_iris(as_frame=True)

In [5]:
# we name all data as "df" because we have no imagination?
# if your notebook is in the same folder as the data file:
# just use the files name.
# df = pd.read_csv("iris.csv")

# or if you data is inside a folder called data, above your working folder:
# you get extra points for that.
#df = pd.read_csv("../data/iris.csv")

# or use an absolute path, that is too ugly, and shows you did not study paths:
#df = pd.read_csv("C:\\Users\\tharg\\......\\.....


In [6]:
df = pd.DataFrame(iris.frame)

In [7]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
# show column names of dataset.
# 5 columns with suboptimal names.
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [9]:
# show data types of dataset.
# four numeric, one categorical variable type.
df.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target                 int32
dtype: object

In [10]:
# simple commnand to show column names and data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


### Short summary notes about the variables.
CustomerID is not important, unless you know who is who.  

Spending score is what makes a huge difference. How could we group the customers, according to their spending score, gender, income, age?   

The second most important variable, in this case, should be annual income.  

Gender might, or might might not make a difference.  

Age is a tricky variable. Depending in the issue in question, it might be in linear correlation with the other variables, or it might grow and then gradually fall in importance. e.g. The possibility of deseases and age should be correlated and should grow as age grows. On the other hand, income could grow as age grows but then, after retirement, as age grows income might fall. The same line of thought might apply to age and spending score.   

Conclusion, we want to group customers in clusters. We examine the variables and their relationships. Finally, we separate the customes in clusters.

# C. Descriptive statistics (DStats) and Exploratory Data Analysis (EDA)

## C.1. Descriptive Statistics (DStats)

In [11]:
# very basic DStats
df.describe()  # show only numeric variables

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [12]:
# Show desc. Stats by class
df.groupby("target").describe().T

target                           0          1          2
sepal length (cm) count  50.000000  50.000000  50.000000
                  mean    5.006000   5.936000   6.588000
                  std     0.352490   0.516171   0.635880
                  min     4.300000   4.900000   4.900000
                  25%     4.800000   5.600000   6.225000
                  50%     5.000000   5.900000   6.500000
                  75%     5.200000   6.300000   6.900000
                  max     5.800000   7.000000   7.900000
sepal width (cm)  count  50.000000  50.000000  50.000000
                  mean    3.428000   2.770000   2.974000
                  std     0.379064   0.313798   0.322497
                  min     2.300000   2.000000   2.200000
                  25%     3.200000   2.525000   2.800000
                  50%     3.400000   2.800000   3.000000
                  75%     3.675000   3.000000   3.175000
                  max     4.400000   3.400000   3.800000
petal length (cm) count  50.000000  50.000000  50.000000
                  mean    1.462000   4.260000   5.552000
                  std     0.173664   0.469911   0.551895
                  min     1.000000   3.000000   4.500000
                  25%     1.400000   4.000000   5.100000
                  50%     1.500000   4.350000   5.550000
                  75%     1.575000   4.600000   5.875000
                  max     1.900000   5.100000   6.900000
petal width (cm)  count  50.000000  50.000000  50.000000
                  mean    0.246000   1.326000   2.026000
                  std     0.105386   0.197753   0.274650
                  min     0.100000   1.000000   1.400000
                  25%     0.200000   1.200000   1.800000
                  50%     0.200000   1.300000   2.000000
                  75%     0.300000   1.500000   2.300000
                  max     0.600000   1.800000   2.500000

# D. Apply Algo to the data
## D.1. Define X and Y and Split set.

Note: For simplicity reasons, I do not split the data in train and test set.

In [15]:
# create a matrix (πίνακας) of X features
X = df.iloc[:,:-1]

In [16]:
# show first 3 lines of data
X.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [17]:
y =  df.iloc[:,-1]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
# fit the 5 clusters customized algorithm to the data.
fit_kmeans = init_kmeans.fit(X)

In [ ]:
# create predictions = assign each observation of X to a cluster
predictions = fit_kmeans.predict(X)

In [ ]:
# show the clusters in which each observation has been assigned to
predictions

In [ ]:
# create a new column, d it to the dataframe to show clustersof observations.
df["cluster"] = pd.DataFrame(
    predictions, columns=["cluster"])

# set cluster column values as category data type
df["cluster"] = df["cluster"].astype('category')

#show first 6 rows of observations
df.head(6)

In [ ]:
# count number of observations in each cluster, sorted by number of observations
df['cluster'].value_counts()

In [ ]:
# same as above, different syntax, output sorted by cluster number
df.groupby('cluster').size()

In [ ]:
# Show average of variables my cluster
# This shows some basic descriptive stats about the clusters.
# e.g. cluster 3 has the highest spending score (82) with average income=82k and age=32
# e.g. cluster 0 has almost highest spending score (79) with average income only 26k and age=25
# This IMPORTANT finding confirms the previous insight about the importance of age.
df.groupby('cluster').mean()

In [ ]:
# plot the clusters with their centroids
sns.scatterplot(x="Annual_Income_(k$)", y="Spending_Score", hue="cluster",
                data=df, palette="deep");

sns.scatterplot(x=fit_kmeans.cluster_centers_[:, 0],
                y=fit_kmeans.cluster_centers_[:, 1],
                s=300, c=sns.color_palette("deep", 5),
                label='Centroid', marker='*');

# Put the legend outside of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0);

In [ ]:
# plot the five clusters in 3D space
three_dim_clusters = px.scatter_3d(
    df, x="Annual_Income_(k$)", y="Spending_Score", z="Age", color="cluster",
    height=600, width=600,
    color_discrete_map={0: "blue", 1:"purple", 2:"green", 3:"black", 4:"red"}
).update_layout(margin=dict(t=40, l=40, r=40, b=40))

three_dim_clusters.show()

# save to file, use everywhere with a browser
#three_dim_clusters.write_html("three_dim_customers.html")  

## D.2. Use the elbow method to get a "suggestion" about the number of clusters.  
This should be done first, but for presentation, learning reasons I wanted to show how kmeans is applied in scikit learn. Use all features.

In [ ]:
df.columns

In [ ]:
# create matrix of features to use in algorithm
X = df[["Genre", "Age", "Annual_Income_(k$)", "Spending_Score"]]
# show first two rows.
X.head(2)

In [ ]:
# encode Male and Female to zero and one, to use this feature in the Kmeans algo
# create a new column named Gender and set Male = 1, Female=0
X["Gender"] = pd.get_dummies(X.Genre, drop_first=True, prefix="Genre")
X.head(3)

In [ ]:
# drop the Genre column from X.
X = X.drop("Genre", axis=1)
# show first 3 rows
X.head(3)

### Apply Kmeans with four features and 1 to 10 clusters, to show the elbow method.

In [ ]:
# empty list to contain the score of each cluster.
clustering_score = []

# loop from 1 to 10 clusters:
for clusters_number in range(1, 11):
    # initiate and customize algo
    kmeans_algo = KMeans(n_clusters=clusters_number, random_state=0)
    # fit algo to the data
    kmeans_algo.fit(X)
    # inertia_ = Sum of squared distances of samples to their closest cluster center.
    # this is the Within Clusters Sum of Squares metric, WCSS
    clustering_score.append(kmeans_algo.inertia_)
   

In [ ]:
# show the list with the clusters' score from 1 to 10
clustering_score

In [ ]:
# WCSS score for 1 cluster
clustering_score[0]  # first item in the list

In [ ]:
# WCSS score for 2 clusters
clustering_score[1]  # second item in the list

In [ ]:
# show the cluster score for the fifth element (5 clusters)
clustering_score[4]

In [ ]:
# creata pandas dataframe from list of clustering WCSS score
wcss_df = pd.DataFrame({'clusters_score':clustering_score})

In [ ]:
# creata new column of dataframe from range of clusters
wcss_df["clusters_number"] = pd.Series(range(1,11))

In [ ]:
#show clusters score for number of clusters
wcss_df

We see that from 4 to five clusters there is a large reduction in WCSS, but from 5 to 6 there is a reduction from 58348 to 51132, much smaller. The reduction in WCSS from 6 to 7 clusters is similar.
There is a much smaller reduction from 7 to 8 clusters.

In [ ]:
# plot WCSS
# create a figure and set size
plt.figure(figsize=(7,5))

#plot the simple range from 1 to 10 on x and the clustering score on y.
plt.plot(range(1, 11), clustering_score)

#plot a red o on number 5 on x axis, and the WCSS score 
plt.scatter(5, clustering_score[4], s = 50, c = 'red', marker='o')

# add title and labels
plt.title('The Elbow Method')
plt.xlabel('No. of Clusters')
plt.ylabel('Clustering Score, WCSS');

We also see that the slope of the curve from 5 to 6 clusters is reduced but very slightly. This suggests five clusters are a good choice.
But using four features instead of two suggest that 6 cluster might also be a good choice.

## D.3. Apply silhouette method to get a suggestion for the number of clusters.  
scikit-learn silhouette score [function and examples.](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

For a more advanced reading on silhouette score you may read this [tutorial.](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py)   

For more advanced clustering performace evaluation methods, you may read this [guide.](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation)

In [ ]:
#create empty list to store score
silhouette_scores_list = []

# loop from 2 to 10 clusters:
for clusters_number in range(2, 11):
    # initiate and customize algo
    kmeans_algo = KMeans(n_clusters=clusters_number, random_state=0)
    # fit algo to the data
    clusters = kmeans_algo.fit_predict(X)
    # calculate silhouette score for each number of clusters
    silhouette_metric_score = silhouette_score(X, clusters)
    # append each score to the list
    silhouette_scores_list.append(silhouette_metric_score)
    print(
        f"For clusters={clusters_number}, the average silhouette_score is: {silhouette_metric_score}")

We get a higher silhouette score for 5 clusters. But only by a tiny difference.

## D.4. Apply KMeans with four features and six clusters

In [ ]:
# our data until now, grouped in 5 clusters
df.head(3)

In [ ]:
#show the features matrix
X.head(3)
# remember we have encoded "Genre" as "Gender".

In [ ]:
kmeans_algo = KMeans(n_clusters=6, random_state=0)
# fit algo to the data
six_clusters_grouping = kmeans_algo.fit_predict(X)

In [ ]:
# create a new column, add it to the dataframe to show cluster sof observations.
df["six_clusters"] = pd.DataFrame(
    six_clusters_grouping, columns=["six_clusters"])

# set six clusters column values as category data type
df["six_clusters"] = df["six_clusters"].astype('category')

In [ ]:
# show first 4 rows
df.head(4)

In [ ]:
# plot the six clusters in the 3D space
three_dim_six_clusters = px.scatter_3d(
    df, x="Annual_Income_(k$)", y="Spending_Score", z="Age",
    color="six_clusters", height=600, width=600,
    color_discrete_map={
        0: "blue", 1:"purple", 2:"green", 3:"black", 4:"red", 5:"brown"}
).update_layout(margin=dict(t=50, l=60, r=60, b=40))

three_dim_six_clusters.show()

# save to file, use everywhere with a browser
#three_dim_clusters.write_html("three_dim_customers.html")  

Examining the data, we see that the customers with spending score 40 to 60 have been devided to two clusters, based on their age. The new cluster mainly contains customers above 40+ with high spending score. It is designated as cluster 1 in this case, and this the larger cluster.  
**Important reminder**: in the end, the number of clusters is also a matter of policy decisions.

In [ ]:
# show clusters by size
df.groupby('cluster').size()

In [ ]:
# show first 5 rows of observations that belong to cluster 1.
df[df["six_clusters"] == 1].head()

In [ ]:
# show basic descriptive stats for cluster labeled as 1.
df[df["six_clusters"] == 1].describe()

In [ ]:
# group cluster labeled as 1, by gender and show size
df[df["six_clusters"] == 1].groupby("Genre").size()

### "Predict" the cluster for one or more new observations.

In [ ]:
## TODO

#### Extra reading about KMeans, silhouette and WCSS metrics.  
“Give a man a fish and you feed him for a day. Teach him how to fish and you feed him for a lifetime”, quote from the Chinese philosopher Lao Tzu, founder of Taoism.  
How to destroy a proverb, by Thanasis Argyriou:  
“Give a student a coding script and you solve his problems for an exam. Teach him how to google and copy-paste and you ***might*** solve his problems for a couple of months.”